<a href="https://colab.research.google.com/github/Sarahkh4/AI_Genesis/blob/main/Ai_Genesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-google-genai pymongo gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-

In [2]:
import os

# Set Gemini API key
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"  # replace with your Gemini key


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

# Set up the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5)

# Prompt Template
prompt_template = PromptTemplate(
    input_variables=["topic"],
    template="""
You are an educational expert. Create a detailed, step-by-step learning path for the topic "{topic}".
Include stages, learning goals, and free resources like books, websites, or courses.
Present it in a clear, structured format.
"""
)


In [4]:
from pymongo import MongoClient

use_mongodb = True  # Set to False if you don't want to use MongoDB

if use_mongodb:
    mongo_client = MongoClient("YOUR_MONGODB_CONNECTION_STRING")
    db = mongo_client["learning_paths_db"]
    collection = db["paths"]

def generate_learning_path(topic):
    prompt = prompt_template.format(topic=topic)
    response = llm.invoke(prompt)

    learning_path = response.content if hasattr(response, "content") else str(response)

    if use_mongodb:
        collection.insert_one({
            "topic": topic,
            "learning_path": learning_path
        })

    return learning_path


In [5]:
import gradio as gr

def chatbot(topic):
    if not topic.strip():
        return "Please enter a topic."
    return generate_learning_path(topic)

def download_learning_path(topic):
    plan = generate_learning_path(topic)
    filename = f"{topic}_learning_path.txt"
    with open(filename, "w") as f:
        f.write(plan)
    return filename

with gr.Blocks() as demo:
    gr.Markdown("## 📘 Gemini AI Learning Path Generator")

    with gr.Row():
        topic_input = gr.Textbox(label="Enter a Topic")
        generate_btn = gr.Button("Generate Learning Path")

    output = gr.Textbox(label="Learning Path", lines=20)
    download_btn = gr.Button("Download")

    generate_btn.click(chatbot, inputs=topic_input, outputs=output)
    download_btn.click(download_learning_path, inputs=topic_input, outputs=None)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://015fa4a371244d7558.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
